In [23]:
#read pdf file 
import PyPDF2

pdf_obj=PyPDF2.PdfReader('./data/HR Policy Manual.pdf')

data=[]
for page in pdf_obj.pages:
  page_text=page.extract_text()
  data.append(page_text)

# print('\n'.join(data))

In [39]:
from langchain.docstore.document import Document
docs=Document(page_content='\n'.join(data))
docs

Document(page_content="HR Policy Manu al \n \nTable of Contents:  \n \n• Introduction  \n• Equal Employment Opportunity  \n• Anti-Harassment and Anti -Discrimination  \n• Employment Status and Records  \n• Work Hours and Attendance  \n• Leaves of Absence  \n• Compensation and Benefits  \n• Performance Management  \n• Employee Conduct and Discip linary Actions  \n• Termination of Employment  \n• Grievance Procedure  \n• Confidentiality and Data Protection  \n• Employee Resources and Support  \n• Amendments to the Policy Manual  \n• Conclusion  \n \nIntroduction  \nWelcome to the HR Policy Manual! This manual provides important guidelines and policies for \nemployees. It aims to ensure a fair, respectful, and productive work environment. Please review this \nmanual carefully and contact the HR department if you have any questions or need further \nclarification.  \n \nEqual Employment Opportunity  \nOur company is committed to providing equal employment opportunities to all individuals. 

# chunking 

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [40]:
#method-1
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
split_text=text_splitter.split_text('\n'.join(data))
texts=text_splitter.create_documents(split_text)
len(texts)

7

In [41]:
#method-2
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
texts=text_splitter.split_documents([docs])
len(texts)

7

## embedding instance

In [72]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from dotenv import load_dotenv

load_dotenv()

embdd=SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-V2')
embdd.client.get_sentence_embedding_dimension()

384

## vector store pinecone

In [88]:
from langchain.vectorstores import Pinecone
from pinecone import Pinecone as pc, PodSpec

load_dotenv()
#create new index and deleting prev
def store_embeddings(chunk_text,embeddings):
  pc_config=pc()
  index_name='pdf-store'

  for name in pc_config.list_indexes().names():
    if name!=index_name:
      try:
        pc_config.delete_index(name)
        print(f'delete index {index_name}')
      except Exception as e:
        print('no index is there')
  
  if pc_config.list_indexes().names()==[]:
    print('creating new index')
    pc_config.create_index(name=index_name,
                    dimension=embeddings.client.get_sentence_embedding_dimension(),
                    metric='dotproduct',
                    spec=PodSpec(environment='gcp-starter'))

      
  Pinecone.from_documents(chunk_text,embeddings,index_name=index_name)

In [89]:
store_embeddings(texts,embdd)